# Create the regionalized version of EF 3.1 in brightway

In [1]:
import bw2data as bd
import numpy as np
import pandas as pd
from mescal import *

In [2]:
bd.projects.set_current(f'ecoinvent3.9.1')

In [3]:
ef_cfs = pd.read_excel('lcia/EF-LCIAMethod_CF(EF-v3.1).xlsx', sheet_name='lciamethods_CF')
ef_biosphere_mapping = pd.read_csv('lcia/EF31_biosphere_mapping.csv')
# TODO: map land occupation flows 

In [4]:
def mapping_reformat(row):
    if row.biosphere.startswith('Transformation, '):
        return 'transformation, ' + row.ef
    elif row.biosphere.startswith('Occupation, '):
        return 'occupation, ' + row.ef
    else:
        return row.ef

In [5]:
ef_biosphere_mapping['ef'] = ef_biosphere_mapping.apply(mapping_reformat, axis=1)

In [6]:
ef_biosphere_mapping_dict = ef_biosphere_mapping.set_index('ef').to_dict()['biosphere']

## Create the new methods

In [7]:
def ecoinvent_compatible_categories(name, cat0, cat1, cat2):
    
    if cat0 == 'Emissions':
        cat1 = cat1.replace('Emissions to ', '')
        cat2 = cat2.replace('Emissions to ', '')
        cat2 = cat2.replace('and', '+')
        
        if cat2 == 'air, unspecified':
            cat2 = None
        elif cat2 == 'air, unspecified (long-term)':
            cat2 = 'low population density, long-term'
            
        if '(biogenic)' in name:
            name = name.replace(' (biogenic)', ', non-fossil')
        elif '(fossil)' in name:
            name = name.replace(' (fossil)', ', fossil')
        elif '(land use change)' in name:
            name = name.replace(' (land use change)', ', to soil or biomass stock')
        
        if 'Particles' in name:
            name.replace('Particles', 'Particulate Matter, ')
            
            if '(PM10)' in name:
                name = name.replace('(PM10)', '> 2.5 um and < 10um')
            elif '(PM0.2 - PM2.5)' in name:
                name = name.replace('(PM0.2 - PM2.5)', '< 2.5 um')
    
    elif cat0 == 'Land use':
        cat1 = cat1.replace('Land ', '')
        name = f'{cat1}, {name}'
        cat1 = 'natural resource'
        cat2 = 'land'
    
    elif cat0 == 'Resources':
        cat2 = f"in {cat1.replace('Resources from ', '')}"
        cat1 = 'natural resource'
    
    else:
        raise ValueError(f"Category0 {cat0} not recognized")
    
    if name in ef_biosphere_mapping_dict.keys():
        name = ef_biosphere_mapping_dict[name]
    
    return name, cat0, cat1, cat2

In [8]:
ef_cfs[['FLOW_name', 'FLOW_class0', 'FLOW_class1', 'FLOW_class2']] = ef_cfs.apply(lambda x: ecoinvent_compatible_categories(x['FLOW_name'], x['FLOW_class0'], x['FLOW_class1'], x['FLOW_class2']), axis=1, result_type='expand')

In [9]:
biosphere_db_spatialized = Database('biosphere3_spatialized_flows')
biosphere_db = Database('biosphere3')

Getting activity data


100%|██████████| 62222/62222 [00:00<00:00, 127804.27it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


...

Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 4718/4718 [00:00<?, ?it/s]

Loaded biosphere3 from brightway!


In [10]:
biosphere_db_dict_name = biosphere_db.list_to_dict('name', 'biosphere')
biosphere_db_spatialized_dict_name = biosphere_db_spatialized.list_to_dict('name', 'biosphere')

In [11]:
biosphere_db_dict_code = biosphere_db.list_to_dict('code', 'biosphere')
biosphere_db_spatialized_dict_code = biosphere_db_spatialized.list_to_dict('code', 'biosphere')

In [12]:
def create_cf_list(df):
    
    reg_cfs = []
    unmapped = []
    
    for i in range(len(df)):
        
        flow_name = df.loc[i, 'FLOW_name']
        flow_name = flow_name.lower()
        
        flow_location = df.loc[i, 'LCIAMethod_location']
        
        flow_cf = df.loc[i, 'CF EF3.1']
        
        if flow_location is np.nan:
            spatialized_db = False
        else:
            flow_name = f'{flow_name}, {flow_location}'
            spatialized_db = True
        
        category1 = df.loc[i, 'FLOW_class1']
        category2 = df.loc[i, 'FLOW_class2']
        
        flow_name = flow_name[0].upper() + flow_name[1:]
        
        if category2 is not None:
            
            if spatialized_db:
                if (flow_name, (category1, category2), 'biosphere3_spatialized_flows') in biosphere_db_spatialized_dict_name.keys():
                    flow_code = biosphere_db_spatialized_dict_name[(flow_name, (category1, category2), 'biosphere3_spatialized_flows')]['code']
                    reg_cfs.append((('biosphere3_spatialized_flows', flow_code), flow_cf))
                else:
                    unmapped.append(('biosphere3_spatialized_flows', flow_name, (category1, category2)))
            
            else:
                if (flow_name, (category1, category2), 'biosphere3') in biosphere_db_dict_name.keys():
                    flow_code = biosphere_db_dict_name[(flow_name, (category1, category2), 'biosphere3')]['code']
                    reg_cfs.append((('biosphere3', flow_code), flow_cf))
                else:
                    unmapped.append(('biosphere3', flow_name, (category1, category2)))
        
        else:
            
            if spatialized_db:
                if (flow_name, (category1,), 'biosphere3_spatialized_flows') in biosphere_db_spatialized_dict_name.keys():
                    flow_code = biosphere_db_spatialized_dict_name[(flow_name, (category1,), 'biosphere3_spatialized_flows')]['code']
                    reg_cfs.append((('biosphere3_spatialized_flows', flow_code), flow_cf))
                else:
                    unmapped.append(('biosphere3_spatialized_flows', flow_name, (category1,)))
            
            else:
                if (flow_name, (category1,), 'biosphere3') in biosphere_db_dict_name.keys():
                    flow_code = biosphere_db_dict_name[(flow_name, (category1,), 'biosphere3')]['code']
                    reg_cfs.append((('biosphere3', flow_code), flow_cf))
                else:
                    unmapped.append(('biosphere3', flow_name, (category1,)))
    
    return reg_cfs, unmapped

In [13]:
def method_names_bw(cat):
    
    # Changes in categories names 
    cat_bw = cat.replace('EF-', '')
    cat_bw = cat_bw.replace('Resource use, fossils', 'Energy resources: non-renewable')
    cat_bw = cat_bw.replace('Resource use, minerals and metals', 'Material resources: metals/minerals')
    cat_bw = cat_bw.replace('Eutrophication marine', 'eutrophication: marine')
    cat_bw = cat_bw.replace('Matter', 'matter formation')
    cat_bw = cat_bw.replace('cancer', 'carcinogenic')
    cat_bw = cat_bw.replace('ozone', 'oxidant')
    cat_bw = cat_bw.replace('-', ': ')
    cat_bw = cat_bw.replace(' : ', ':')
    cat_bw = cat_bw.replace(',', ':')
    cat_bw = cat_bw.replace('_', ', ')
    cat_bw = cat_bw.replace('non: ', 'non-')
    
    return cat_bw

In [14]:
ef_units = {
    'acidification': 'mol H+-Eq',
    'climate change': 'kg CO2-Eq',
    'climate change: biogenic': 'kg CO2-Eq',
    'climate change: fossil': 'kg CO2-Eq',
    'climate change: land use and land use change': 'kg CO2-Eq',
    'ecotoxicity: freshwater': 'CTUe',
    'ecotoxicity: freshwater, inorganics': 'CTUe',
    'ecotoxicity: freshwater, organics': 'CTUe',
    'energy resources: non-renewable': 'MJ, net calorific value',
    'eutrophication: freshwater': 'kg P-Eq',
    'eutrophication: marine': 'kg N-Eq',
    'eutrophication: terrestrial': 'mol N-Eq',
    'human toxicity: carcinogenic': 'CTUh',
    'human toxicity: carcinogenic, inorganics': 'CTUh',
    'human toxicity: carcinogenic, organics': 'CTUh',
    'human toxicity: non-carcinogenic': 'CTUh',
    'human toxicity: non-carcinogenic, inorganics': 'CTUh',
    'human toxicity: non-carcinogenic, organics': 'CTUh',
    'ionising radiation: human health': 'kBq U235-Eq',
    'land use': 'dimensionless',
    'material resources: metals/minerals': 'kg Sb-Eq',
    'ozone depletion': 'kg CFC-11-Eq',
    'particulate matter formation': 'disease incidence',
    'photochemical oxidant formation: human health': 'kg NMVOC-Eq',
    'water use': 'm3 world eq. deprived',
}

In [15]:
ef_methods_bw = [i for i in bd.methods if i[0] == 'EF v3.1']
regionalized_methods = {}
unlinked = []
methods_to_regionalize = [
    'acidification',
    'eutrophication, terrestrial',
    'land use',
    'water use',
]

for i in range(len(ef_methods_bw)):
    method = bd.Method(ef_methods_bw[i])
    method_name = ef_methods_bw[i][1]
    method_name = method_names_bw(method_name)
    cfs = method.load()
    regionalized_cfs = []
    
    for j in cfs:
        
        if method_name not in methods_to_regionalize:
            regionalized_cfs.append(j)
        
        else:
            flow = biosphere_db_dict_code[(j[0][0], j[0][1])]
            flow_name = flow['name'].lower()
            flow_category_1 = flow['categories'][0]
            if len(flow['categories']) == 1:
                df_regionalized_flows = ef_cfs[
                    (ef_cfs['FLOW_name'].str.lower() == flow_name) 
                    & (ef_cfs['FLOW_class1'] == flow_category_1)
                    & (ef_cfs['FLOW_class2'].isna())
                    & (ef_cfs['LCIAMethod_name'].str.lower() == method_name)
                ].reset_index()
                
                if len(df_regionalized_flows) == 0:
                    unlinked.append(('biosphere3', flow_name, (flow_category_1,)))
                
            else:
                flow_category_2 = flow['categories'][1]
            
                df_regionalized_flows = ef_cfs[
                    (ef_cfs['FLOW_name'].str.lower() == flow_name) 
                    & (ef_cfs['FLOW_class1'] == flow_category_1)
                    & (ef_cfs['FLOW_class2'] == flow_category_2)
                    & (ef_cfs['LCIAMethod_name'].str.lower() == method_name)
                ].reset_index()
                
                if len(df_regionalized_flows) == 0:
                    unlinked.append(('biosphere3', flow_name, (flow_category_1, flow_category_2)))
            
            new_regionalized_cfs, new_unlinked = create_cf_list(df_regionalized_flows)
            regionalized_cfs += new_regionalized_cfs
            unlinked += new_unlinked
    
    regionalized_methods[method_name] = regionalized_cfs

In [17]:
list(set([i[1] for i in unlinked if 'biosphere3' == i[0]]))

['occupation, grassland, natural, for livestock grazing',
 'occupation, permanent crop, non-irrigated, extensive',
 'transformation, from dump site, residual material landfill',
 'water',
 'transformation, from dump site, sanitary landfill',
 'occupation, annual crop, non-irrigated, intensive',
 'occupation, permanent crop',
 'occupation, unspecified, natural (non-use)',
 'transformation, from dump site',
 'transformation, from dump site, inert material landfill',
 'occupation, urban, green area',
 'transformation, to annual crop',
 'occupation, pasture, man made',
 'transformation, to dump site, residual material landfill',
 'occupation, heterogeneous, agricultural',
 'occupation, annual crop, greenhouse',
 'occupation, permanent crop, irrigated, extensive',
 'occupation, permanent crop, non-irrigated',
 'transformation, from annual crop',
 'occupation, shrub land, sclerophyllous',
 'transformation, to dump site, inert material landfill',
 'transformation, from unknown',
 'occupation,

In [19]:
# write = False
# impact_category_list = list(ef_cfs.LCIAMethod_name.unique())
# ef_regionalized_cfs = {}
# not_mapped = {}
# for cat in impact_category_list:
#     ef_regionalized_method = bd.Method(('EF v3.1 regionalized', cat.lower()))
#     ef_regionalized_method_metadata = {'unit': ef_units[cat.lower()]}
#     ef_regionalized_method.register(**ef_regionalized_method_metadata)
#     ef_regionalized_method.write(new_ef_regionalized_cfs)